# Day 3: Crossed Wires

## Part 1

The gravity assist was successful, and you're well on your way to the Venus refuelling station. During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.

Opening the front panel reveals a jumble of wires. Specifically, **two wires** are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to **find the intersection point closest to the central port**. Because the wires are on a grid, use the [Manhattan distance](https://en.wikipedia.org/wiki/Taxicab_geometry) for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

For example, if the first wire's path is `R8,U5,L5,D3`, then starting from the central port (`o`), it goes right `8`, up `5`, left `5`, and finally down `3`:

```
...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........
```

Then, if the second wire's path is `U7,R6,D4,L4`, it goes up `7`, right `6`, down `4`, and left `4`:

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........

```

These wires cross at two locations (marked X), but the lower-left one is closer to the central port: its distance is `3 + 3 = 6`.

Here are a few more examples:

- `R75,D30,R83,U83,L12,D49,R71,U7,L72`<br>
  `U62,R66,U55,R34,D71,R55,D58,R83` = distance `159`
- `R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51`<br>
  `U98,R91,D20,R16,D67,R40,U7,R15,U6,R7` = distance `135`

**What is the Manhattan distance** from the central port to the closest intersection?

### Extract data

In [1]:
from collections import namedtuple

Instruction = namedtuple('Instruction', 'operation value')

data = []

with open('./data/input', 'r') as f:
    # Wire 1
    wire1 = [Instruction(instruction[0], int(instruction[1:])) for instruction in f.readline().rstrip().split(',')]
    data.append(wire1)
    # Wire 2
    wire2 = [Instruction(instruction[0], int(instruction[1:])) for instruction in f.readline().rstrip().split(',')]
    data.append(wire2)
    
print(data[0][:5], data[1][:5])

[Instruction(operation='R', value=1000), Instruction(operation='D', value=940), Instruction(operation='L', value=143), Instruction(operation='D', value=182), Instruction(operation='L', value=877)] [Instruction(operation='L', value=990), Instruction(operation='D', value=248), Instruction(operation='L', value=833), Instruction(operation='U', value=137), Instruction(operation='L', value=556)]


In [2]:
sample_data = []

wire1 = [Instruction(instruction[0], int(instruction[1:])) 
         for instruction in 'R8,U5,L5,D3'.split(',')]
wire2 = [Instruction(instruction[0], int(instruction[1:])) 
         for instruction in 'U7,R6,D4,L4'.split(',')]
sample_data.append(wire1)
sample_data.append(wire2)

sample_data

[[Instruction(operation='R', value=8),
  Instruction(operation='U', value=5),
  Instruction(operation='L', value=5),
  Instruction(operation='D', value=3)],
 [Instruction(operation='U', value=7),
  Instruction(operation='R', value=6),
  Instruction(operation='D', value=4),
  Instruction(operation='L', value=4)]]

In [3]:
sample_data2 = []

wire1 = [Instruction(instruction[0], int(instruction[1:])) 
         for instruction in 'R75,D30,R83,U83,L12,D49,R71,U7,L72'.split(',')]
wire2 = [Instruction(instruction[0], int(instruction[1:])) 
         for instruction in 'U62,R66,U55,R34,D71,R55,D58,R83'.split(',')]
sample_data2.append(wire1)
sample_data2.append(wire2)

sample_data2

[[Instruction(operation='R', value=75),
  Instruction(operation='D', value=30),
  Instruction(operation='R', value=83),
  Instruction(operation='U', value=83),
  Instruction(operation='L', value=12),
  Instruction(operation='D', value=49),
  Instruction(operation='R', value=71),
  Instruction(operation='U', value=7),
  Instruction(operation='L', value=72)],
 [Instruction(operation='U', value=62),
  Instruction(operation='R', value=66),
  Instruction(operation='U', value=55),
  Instruction(operation='R', value=34),
  Instruction(operation='D', value=71),
  Instruction(operation='R', value=55),
  Instruction(operation='D', value=58),
  Instruction(operation='R', value=83)]]

### Calculating Manhattan distance

In [4]:
Point = namedtuple('Point', 'x y')

def calculate_manhattan_distance(p1: Point, p2: Point) -> int:
    return abs(p1.x - p2.x) + abs(p1.y - p2.y)

calculate_manhattan_distance(Point(-1000, -900), Point(-990, -890))

20

### Find the intersections

In [5]:
def _generate_points(current_pos: Point, instruction: Instruction):
    new_points = None
    
    if instruction.operation == 'R':
        new_points = [Point(x, current_pos.y)
                      for x in range(current_pos.x + 1, current_pos.x + instruction.value + 1)]

    elif instruction.operation == 'L':
        new_points = [Point(x, current_pos.y)
                      for x in range(current_pos.x - 1, current_pos.x - instruction.value - 1, -1)]

    elif instruction.operation == 'U':
        new_points = [Point(current_pos.x, y)
                      for y in range(current_pos.y + 1, current_pos.y + instruction.value + 1)]
        
    elif instruction.operation == 'D':
        new_points = [Point(current_pos.x, y)
                      for y in range(current_pos.y - 1, current_pos.y - instruction.value - 1, -1)]
        
    return new_points
    
_generate_points(Point(-5, -10), Instruction('U', 2))

[Point(x=-5, y=-9), Point(x=-5, y=-8)]

In [6]:
from enum import Enum

class WireNumber(Enum):
    WIRE_1 = 0
    WIRE_2 = 1

def generate_wire_map(wire_map: dict, instructions: list[Instruction], wire_nb: WireNumber) -> map:
    current_pos = Point(0, 0)
    new_points = []
    for instruction in instructions:
        new_points = [*new_points, *(_generate_points(current_pos, instruction))]
        current_pos = new_points[-1]
        
    for new_point in new_points:
        if new_point in wire_map.keys():
            wire_map[new_point].add(wire_nb)
        else:
            wire_map[new_point] = {wire_nb}
            
    return wire_map

def calculate_crossing_points(wire_map: dict) -> list[Point]:
    return [point for point, val in wire_map.items() if len(val) == 2]

wire_map = {}
wire_map = generate_wire_map(wire_map, sample_data[0], WireNumber.WIRE_1)
wire_map = generate_wire_map(wire_map, sample_data[1], WireNumber.WIRE_2)
calculate_crossing_points(wire_map)

[Point(x=6, y=5), Point(x=3, y=3)]

### Calculate result

In [7]:
def calculate_res_p1(wire_map: map) -> int:
    crossing_points = calculate_crossing_points(wire_map)
    
    res = min(
        [calculate_manhattan_distance(Point(0, 0), crossing_point)
         for crossing_point in crossing_points]
    )
    return res

In [8]:
wire_map = {}
wire_map = generate_wire_map(wire_map, data[0], WireNumber.WIRE_1)
wire_map = generate_wire_map(wire_map, data[1], WireNumber.WIRE_2)

In [9]:
calculate_res_p1(wire_map)

865

Your puzzle answer was `865`.

## Part 2

It turns out that this circuit is very timing-sensitive; you actually need to **minimize the signal delay**.

To do this, calculate the **number of steps** each wire takes to reach each intersection; choose the intersection where the **sum of both wires' steps** is lowest. If a wire visits a position on the grid multiple times, use the steps value from the **first** time it visits that position when calculating the total value of a specific intersection.

The number of steps a wire takes is the total number of grid squares the wire has entered to get to that location, including the intersection being considered. Again consider the example from above:

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
```

In the above example, the intersection closest to the central port is reached after `8+5+5+2 =`**`20`** steps by the first wire and `7+6+4+3 =`**`20`** steps by the second wire for a total of `20+20 =`**`40`** steps.

However, the top-right intersection is better: the first wire takes only `8+5+2 =`**`15`** and the second wire takes only `7+6+2 =`**`15`**, a total of `15+15 =`**`30`** steps.

Here are the best steps for the extra examples from above:

- `R75,D30,R83,U83,L12,D49,R71,U7,L72`<br>
  `U62,R66,U55,R34,D71,R55,D58,R83 =`**`610`** steps
- `R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51`<br>
  `U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 =`**`410`** steps

**What is the fewest combined steps the wires must take to reach an intersection?**

### Calculate number of steps needed to reach a crossing point

In [10]:
def calculate_nb_of_steps(crossing_point: Point, instructions: list[Instruction]):
    nb_steps = 0
    current_pos = Point(0, 0)
    new_points = []
    for instruction in instructions:
        new_points = _generate_points(current_pos, instruction)
        
        if crossing_point in new_points:
            nb_steps += new_points.index(crossing_point) + 1
            break
            
        nb_steps += instruction.value
        current_pos = new_points[-1]
        
    return nb_steps

wire_1_sample_steps = calculate_nb_of_steps(Point(x=6, y=5), sample_data[0])
wire_2_sample_steps = calculate_nb_of_steps(Point(x=6, y=5), sample_data[1])

print(wire_1_sample_steps, wire_2_sample_steps)

15 15


### Calculate result

In [11]:
def calculate_res_p2(crossing_points: list[Point], instructions: list[list[Instruction]]) -> int:
    res = []
    
    for crossing_point in crossing_points:
        nb_steps = calculate_nb_of_steps(crossing_point, instructions[0])
        nb_steps += calculate_nb_of_steps(crossing_point, instructions[1])
        res.append(nb_steps)
        
    return min(res)

wire_map = {}
wire_map = generate_wire_map(wire_map, sample_data2[0], WireNumber.WIRE_1)
wire_map = generate_wire_map(wire_map, sample_data2[1], WireNumber.WIRE_2)
crossing_points = calculate_crossing_points(wire_map)
calculate_res_p2(crossing_points, sample_data2)

610

In [12]:
wire_map = generate_wire_map(wire_map, data[0], WireNumber.WIRE_1)
wire_map = generate_wire_map(wire_map, data[1], WireNumber.WIRE_2)
crossing_points = calculate_crossing_points(wire_map)

calculate_res_p2(crossing_points, data)

35038

Your puzzle answer was `35038`.